# Learn Convolutional Neural Network with Keras



Code in this notebook obtain from [Keras - Python Deep Learning Neural Network API Course](https://deeplizard.com/learn/playlist/PLZbbT5o_s2xrwRnXk_yCPtnqqo4_u2YGL) or you can watch that course in [Youtube ](https://www.youtube.com/watch?v=qFJeN9V1ZsI&t=2380s)

In [ ]:
import numpy as np
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model

from tensorflow.keras.applications import vgg16
from tensorflow.keras.applications import mobilenet
from tensorflow.keras.applications import imagenet_utils

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

from sklearn.metrics import confusion_matrix

warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

### Set Seed to Reproducible Result

In [ ]:
os.environ['PYTHONHASHSEED'] = '0'
os.environ['CUDA_VISIBLE_DEVICES'] = ''
np.random.seed(27)
random.seed(23)
tf.random.set_seed(2723)

### Extract Zip File

In [ ]:
import zipfile
import glob

zip_file = glob.glob('../input/dogs-vs-cats/*.zip')  
print(zip_file)

def extract_zip(file):
    with zipfile.ZipFile(file,"r") as zip_ref:
        zip_ref.extractall("temp")
        
for files in zip_file:
    extract_zip(files)

### Manage Directory
Split data into training, validating, and testing data. I only took 1000 images for training, 200 images for validation and 100 images for testing. This is to minimize computation because I just want to learn how to make CNN in Keras

In [ ]:
train_path = 'train'
valid_path = 'valid'
test_path = 'test'

if os.path.isdir('train/dog') is False:
    os.makedirs(train_path + '/dog')
    os.makedirs(train_path + '/cat')
    os.makedirs(valid_path + '/dog')
    os.makedirs(valid_path + '/cat')
    os.makedirs(test_path + '/dog')
    os.makedirs(test_path + '/cat')

    for i in random.sample(glob.glob('temp/train/cat*'), 500):
        shutil.move(i, train_path + '/cat')      
    for i in random.sample(glob.glob('temp/train/dog*'), 500):
        shutil.move(i, train_path + '/dog')
    for i in random.sample(glob.glob('temp/train/cat*'), 100):
        shutil.move(i, valid_path + '/cat')        
    for i in random.sample(glob.glob('temp/train/dog*'), 100):
        shutil.move(i, valid_path + '/dog')
    for i in random.sample(glob.glob('temp/train/cat*'), 50):
        shutil.move(i, test_path + '/cat')      
    for i in random.sample(glob.glob('temp/train/dog*'), 50):
        shutil.move(i, test_path + '/dog')

### Load Data
Load data from directory and preprocessing with VGG16. The VGG16 model perfoms preprocessing only by substracting all color channels by their average

In [ ]:
train_batches = (ImageDataGenerator(preprocessing_function=vgg16.preprocess_input)
                     .flow_from_directory(directory=train_path, 
                                          target_size=(224,224), 
                                          classes=['cat', 'dog'], 
                                          batch_size=10))

valid_batches = (ImageDataGenerator(preprocessing_function=vgg16.preprocess_input)
                 .flow_from_directory(directory=valid_path, 
                                      target_size=(224,224), 
                                      classes=['cat', 'dog'], 
                                      batch_size=10))

test_batches = (ImageDataGenerator(preprocessing_function=vgg16.preprocess_input)
                .flow_from_directory(directory=test_path, 
                                     target_size=(224,224), 
                                     classes=['cat', 'dog'], 
                                     batch_size=10, 
                                     shuffle=False))

### Plot Images Function

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
imgs, labels = next(train_batches)
plotImages(imgs)

All images look saturated, this is the effect of VGG16 preprocessing

## Make CNN Model From Scratch
This CNN scratch model has 2 convolutional layers and 2 max polling layers with `relu` activation fucntion. This model have 2 output nodes, this is because our data have two classes, namely dog and cat

In [ ]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding = 'same', input_shape=(224,224,3)),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = 'same'),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Flatten(),
    Dense(units=2, activation='softmax')
])
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001),
             loss="categorical_crossentropy",
             metrics=['accuracy'])

model.fit(x=train_batches,
         validation_data=valid_batches,
         epochs=10,
         verbose=2)

It can be seen that the model has perfect accuracy for training data but only has an accuray 0.61 for validation data so this model is overfitting.

### Try To Predict Test Data

In [ ]:
predictions = model.predict(test_batches)
predictions

The `predict` method returns the probability of an image for each class

### Confusion Matrix

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
predictions_labels = np.argmax(predictions, axis=1)
cm = confusion_matrix(test_batches.classes, predictions_labels)

cm_plot_labels = ['cat','dog']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

## Transfer Learning

Transfer learning is a technique for using pre-trained model and fine-tune those parameter to perform a new task. So, we just load the model with its weights and add a new layers so that model can predict new data that don't seen before. There are so many pre-trained models, I only wanted to use two of them, VGG16 and MobileNet.

### VGG16 Model


In [ ]:
vgg16_model = vgg16.VGG16()
vgg16_model.summary()

VGG16 has more then 138 millions parameters with output layer having 1000 nodes, this is because VGG16 is trained by ImageNet dataset with 1000 labels. I will adjust output layer so that it only has 2 nodes and I will to freeze every layers so the weights don't change while the model is trained

In [ ]:
model = Sequential()
for layer in vgg16_model.layers[:-1]:
    model.add(layer)

for layer in model.layers:
    layer.trainable = False
    
model.add(Dense(units=2, activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(x=train_batches,
          validation_data=valid_batches,
          batch_size=10,
          epochs=3,
          verbose=2
)

It can be seen that with only with 3 epochs, the model has an accuracy of 0.98 for training data and validation data. This model not overfitting like our previous model.

In [ ]:
predictions = model.predict(x=test_batches, verbose=0)
cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(predictions, axis=-1))

cm_plot_labels = ['cat','dog']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

### MobileNet Model
The difference between this model and the VGG16 model is the size of the mode. VGG16 has a size 557MB but MobileNet only has 17MB. Very difference! Like the name, this model is very suitable for use in mobile applications. This model only has 4 million parameters but the accuracy only slightly difference from VGG16

In [ ]:
mobile = mobilenet.MobileNet()
mobile.summary()

I'm not refining the model and just using the specified weights to predict the new image. You can fine-tuned this model like VGG16, as I have done before

In [ ]:
def prepareImage(file):
    img = image.load_img(file, target_size=(224, 224))
    img_array = image.img_to_array(img)
    return mobilenet.preprocess_input(np.expand_dims(img_array, axis=0))

def getRandomImageFile():
    return random.sample(glob.glob('temp/test1/*'), 1)[0]

In [ ]:
img = getRandomImageFile()
img_prepared = prepareImage(img)
plt.imshow(img_prepared[0])

In [ ]:
predictions = mobile.predict(img_prepared)
predictions_label = imagenet_utils.decode_predictions(predictions)
predictions_label

## Image Augmentation
This technique often used to increase the size of training data. This technique will manipulate the original image with zooming, rotating, fliping, etc. Image augmentation is also one way to overcome overfitting

In [ ]:
gen = ImageDataGenerator(rotation_range=10, 
                         width_shift_range=0.1, 
                         height_shift_range=0.1, 
                         shear_range=0.15, 
                         zoom_range=0.1, 
                         channel_shift_range=10.,
                         horizontal_flip=True)

In [ ]:
image_path = getRandomImageFile()
image_random = np.expand_dims(plt.imread(image_path),0)
plt.imshow(image_random[0])

In [ ]:
aug_iter = gen.flow(image_random)
aug_images = [next(aug_iter)[0].astype(np.uint8) for i in range(10)]
plotImages(aug_images)